In [1]:
%load_ext autoreload
%autoreload 2
import tichu_rustipy as tr
import numpy as np
from IPython.display import display, HTML
import pickle

def display_colored_hand(hand_str):
    # Convert ANSI escape codes to HTML
    hand_str = (hand_str
        .replace('\x1b[31m', '<span style="color: red">')
        .replace('\x1b[32m', '<span style="color: green">')
        .replace('\x1b[33m', '<span style="color: yellow">')
        .replace('\x1b[34m', '<span style="color: dodgerblue ">')
        .replace('\x1b[0m', '</span>')
    )
    display(HTML(hand_str))
def save_dict(dictionary, filename):
    with open(filename, 'wb') as file:
        pickle.dump(dictionary, file)

# Loading the dictionary from a file
def load_dict(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [ ]:
db = tr.BSWSimple("../tichu_rust/bsw.db")
display_colored_hand(tr.print_hand(db.get_round(0)[0].first_14))
db.len()

In [2]:
#db_as_np = tr.bulk_transform_db_into_np56_array(db)
#np.save("db_as_np", db_as_np)
db_as_np = np.load("db_as_np.npy")

## Later, when using, need to reshape: db_as_np[0].reshape(4, 14)
print(db_as_np.shape)

(85059740, 56)


In [ ]:
# Calculate labels and mapping from incoming card tuples to index
incoming_card_combination_to_label_num = {}
incoming_card_labels = np.zeros(len(db_as_np), dtype=np.uint16)
i = 0
while i < len(db_as_np):
    prh_round = db.get_round(i//4)
    for j in range(4):
        incoming_card_combo = tr.prh_to_incoming_cards(prh_round[j])
        if not incoming_card_combo in incoming_card_combination_to_label_num:
            incoming_card_combination_to_label_num[incoming_card_combo] = len(incoming_card_combination_to_label_num)
        incoming_card_labels[i+j] = incoming_card_combination_to_label_num[incoming_card_combo]
    i += 4
label_num_to_incoming_card_combination = {value: key for key,value in incoming_card_combination_to_label_num.items()}
# Save
np.save("incoming_card_labels", incoming_card_labels)
save_dict(incoming_card_combination_to_label_num, "incoming_card_combination_to_label_num.pkl")
save_dict(label_num_to_incoming_card_combination, "label_num_to_incoming_card_combination.pkl")

In [3]:
incoming_card_labels = np.load("incoming_card_labels.npy")
incoming_card_combination_to_label_num = load_dict("incoming_card_combination_to_label_num.pkl")
label_num_to_incoming_card_combination = load_dict("label_num_to_incoming_card_combination.pkl")

In [6]:
len(incoming_card_combination_to_label_num)

2389

## Card Game Neural Network Architecture
### Input Processing

+ Input shape: [N_samples, 56] (binary representation of hands)
+ Split into:
  - Regular cards [4, 13] (4 colors × 13 values)
  - Special cards [4]



### Regular Cards Path
#### First Layer: ColorInvariantConv -> explained below

8 types of filters (4 filters each = 32 total):

+ Street detection:
  - (4, 5) -> [1, 9] × 4 = 36 features
  - (4, 6) -> [1, 8] × 4 = 32 features
  - (4, 7) -> [1, 7] × 4 = 28 features
+ Single color patterns:
  - (1, 5) -> [4, 9] × 4 = 144 features
+ Pair street patterns:
  - (4, 2) -> [1, 12] × 4 = 48 features
  - (4, 3) -> [1, 11] × 4 = 44 features
  - (4, 4) -> [1, 10] × 4 = 40 features
+ Value patterns:
  - (4, 1) -> [1, 13] × 4 = 52 features

Total features from regular cards: 424

### Special Cards Path

+ Simple dense layer: 4 -> 16 features

### Two Architecture Options
#### Option 1: Direct Flatten

1. Flatten all ColorInvariantConv outputs
2. Concatenate with special cards features
3. Total features: 424 + 16 = 440
4. Dense layers: 256 -> 128
5. Output layer: [N, 2389]

#### Option 2: Separate Processing

1. Process each filter type through additional Conv1d (16 features each)
2. 8 parallel paths of length 16,  more features
3. Concatenate with special cards features (16)
4. Total features: Not sure, a lot
5. Dense layers: 256 -> 128
6. Output layer: [N, 2389]

### Key Features

+ Color invariance through ColorInvariantConv in first layer
+ Game-specific filter sizes capturing relevant patterns
+ Separate processing of special cards
+ Direct modeling of joint probability distribution over 2389 valid combinations
+ No padding in convolutions to preserve pattern semantics
<img src="./model-comparison.svg" />

In [9]:
import torch
import torch.nn as nn

class ColorInvariantConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size)
        
        # Generate all possible color permutations (4! = 24)
        self.color_perms = list(itertools.permutations(range(4)))
        
    def forward(self, x):
        # x shape: [batch_size, in_channels, 4, 13]
        batch_size = x.size(0)
        
        # Apply convolution to each color permutation
        outputs = []
        for perm in self.color_perms:
            # Permute the colors (dim=2 is the color dimension)
            x_perm = x[:, :, perm, :]
            out = self.conv(x_perm)
            outputs.append(out)
            
        # Average over all permutations
        # Stack along a new dimension and then mean over it
        return torch.stack(outputs).mean(0)

ModuleNotFoundError: No module named 'torch'